In [ ]:
!pip install -U "transformers>=4.34.0" "datasets>=2.15.0" "accelerate>=0.21.0" \
            "torch>=2.0.0" "pandas>=2.0.0" "numpy>=1.24.0" "scikit-learn>=1.3.0" \
            "joblib>=1.3.0" "pyarrow>=14.0.0" "fastparquet>=2023.8.0" "tqdm>=4.66.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 105.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from datasets import Dataset, load_dataset
from transformers import AutoTokenizer
import pandas as pd

def load_parquet_dataset(parquet_path, text_field="code", label_field="label"):
    """
    Load a parquet file as a streaming Hugging Face dataset.
    Returns an IterableDataset that can be sampled from.
    """
    dataset = load_dataset("parquet", data_files=parquet_path, split="train", streaming=True)
    return dataset.map(lambda x: {text_field: x[text_field], label_field: x[label_field]})


class CodeDatasetPreprocessor:
    """
    Tokenizes code or text fields in Hugging Face datasets for transformer models.
    """

    def __init__(self, model_name_or_path, max_length=512, text_field="code"):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)
        self.max_length = max_length
        self.text_field = text_field

    def tokenize_batch(self, examples):
        return self.tokenizer(
            examples[self.text_field],
            truncation=True,
            padding="max_length",
            max_length=self.max_length
        )
    def prepare(self, dataset):
        """
        Apply tokenization to a Hugging Face Dataset (non-streaming).
        """
        return dataset.map(
            self.tokenize_batch,
            batched=True,
            remove_columns=[self.text_field]
        )


In [ ]:
# src/metrics.py
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import math
from typing import Dict

def classification_metrics(preds, labels) -> Dict:
    """Compute accuracy, precision, recall, and f1 (weighted).
    preds can be logits (ndarray) or 1d predicted labels."""
    import numpy as _np
    preds_arr = _np.array(preds)
    if preds_arr.ndim > 1:
        pred_labels = preds_arr.argmax(axis=1)
    else:
        pred_labels = preds_arr
    acc = accuracy_score(labels, pred_labels)
    prec, rec, f1, _ = precision_recall_fscore_support(labels, pred_labels, average='weighted', zero_division=0)
    return {"accuracy": acc, "precision": prec, "recall": rec, "f1": f1}

def perplexity(loss):
    return math.exp(loss)

In [ ]:
# src/models.py
from transformers import AutoModelForSequenceClassification, AutoConfig

def get_sequence_classification_model(name_or_path, num_labels, from_pretrained=True):
    if from_pretrained:
        model = AutoModelForSequenceClassification.from_pretrained(name_or_path, num_labels=num_labels)
    else:
        config = AutoConfig.from_pretrained(name_or_path, num_labels=num_labels)
        model = AutoModelForSequenceClassification.from_config(config)
    return model

In [ ]:
# ==============================
# ✅ SETUP
# ==============================
!pip install -q transformers datasets accelerate evaluate scikit-learn

import os
import numpy as np
import pandas as pd
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
)
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import torch, gc


# ==============================
# ✅ DATA HANDLING
# ==============================
def load_parquet_dataset(parquet_path, text_field="code", label_field="label"):
    dataset = load_dataset("parquet", data_files=parquet_path)["train"]
    sample = dataset[0]

    if text_field not in sample:
        raise ValueError(f"'{text_field}' not found in dataset columns: {list(sample.keys())}")
    if label_field not in sample:
        raise ValueError(f"'{label_field}' not found in dataset columns: {list(sample.keys())}")

    return dataset


def subsample_dataset(dataset, max_size=20000, seed=42):
    """Keep manageable subset (for Colab GPU)."""
    if len(dataset) > max_size:
        dataset = dataset.shuffle(seed=seed).select(range(max_size))
    return dataset


# ==============================
# ✅ TOKENIZATION
# ==============================
class CodeDatasetPreprocessor:
    def __init__(self, model_name_or_path, max_length=512, text_field="code"):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)
        self.max_length = max_length
        self.text_field = text_field

    def tokenize_batch(self, examples):
        return self.tokenizer(
            examples[self.text_field],
            truncation=True,
            max_length=self.max_length,
        )

    def prepare(self, dataset):
        return dataset.map(
            self.tokenize_batch,
            batched=True,
            remove_columns=[self.text_field]
        )


# ==============================
# ✅ METRICS (BINARY)
# ==============================
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)

    acc = accuracy_score(labels, preds)
    prec, rec, f1, _ = precision_recall_fscore_support(
        labels, preds, average="binary", zero_division=0
    )

    return {
        "accuracy": acc,
        "precision": prec,
        "recall": rec,
        "f1": f1,
    }


# ==============================
# ✅ MAIN TRAINING FUNCTION (BINARY)
# ==============================
def main():
    model_name = "microsoft/codebert-base"
    train_path = "/content/drive/MyDrive/train.parquet"
    val_path = "/content/drive/MyDrive/val.parquet"
    out_dir = "/content/drive/MyDrive/experiments_binary"

    num_labels = 2            # ✅ BINARY
    epochs = 6               # ✅ tuned for accuracy
    batch_size = 8
    grad_accum = 4
    lr = 1.5e-5

    os.makedirs(out_dir, exist_ok=True)

    print("💻 Device:", "cuda" if torch.cuda.is_available() else "cpu")

    # ==============================
    # ✅ LOAD DATA
    # ==============================
    print("📂 Loading parquet datasets...")
    train_ds = load_parquet_dataset(train_path)
    val_ds = load_parquet_dataset(val_path)

    print("📉 Subsampling datasets (if large)...")
    train_ds = subsample_dataset(train_ds, max_size=40000)
    val_ds = subsample_dataset(val_ds, max_size=4000)

    # ==============================
    # ✅ TOKENIZATION
    # ==============================
    print("🔧 Tokenizing datasets...")
    preproc = CodeDatasetPreprocessor(model_name, max_length=256, text_field="code")
    train_tok = preproc.prepare(train_ds)
    val_tok = preproc.prepare(val_ds)

    # ==============================
    # ✅ MODEL
    # ==============================
    print("🤖 Initializing model...")
    model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=num_labels
    )

    model.gradient_checkpointing_enable()

    # ==============================
    # ✅ TRAINING ARGS
    # ==============================
    args = TrainingArguments(
        output_dir=out_dir,
        eval_strategy="epoch",
        save_strategy="epoch",
        learning_rate=lr,

        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,

        num_train_epochs=epochs,
        gradient_accumulation_steps=grad_accum,

        weight_decay=0.05,
        logging_steps=200,

        fp16=True,
        save_total_limit=2,

        load_best_model_at_end=True,
        metric_for_best_model="f1",
        greater_is_better=True,

        report_to="none",
        lr_scheduler_type="cosine",
        warmup_ratio=0.1,
        optim="adamw_torch",
        dataloader_num_workers=2,
        push_to_hub=False,
    )

    data_collator = DataCollatorWithPadding(tokenizer=preproc.tokenizer)

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=train_tok,
        eval_dataset=val_tok,
        tokenizer=preproc.tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )
    # ==============================
    # ✅ MEMORY CLEANUP
    # ==============================
    gc.collect()
    torch.cuda.empty_cache()

    # ==============================
    # ✅ TRAIN
    # ==============================
    print("🚀 Starting binary fine-tuning with CodeBERT...")
    trainer.train()

    trainer.save_model(out_dir)
    preproc.tokenizer.save_pretrained(out_dir)

    print(f"✅ Training complete! Best binary model saved to {out_dir}")


if __name__ == "__main__":
    main()


💻 Device: cuda
📂 Loading parquet datasets...
📉 Subsampling datasets (if large)...
🔧 Tokenizing datasets...


Map:   0%|          | 0/40000 [00:00<?, ? examples/s]

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

🤖 Initializing model...


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-753148683.py:171: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


🚀 Starting binary fine-tuning with CodeBERT...


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.078600,0.077312,0.977500,0.989776,0.967174,0.978345
2,0.041900,0.053354,0.984750,0.989448,0.981446,0.985431
3,0.032400,0.053568,0.987250,0.989032,0.986679,0.987854
4,0.028400,0.043491,0.989000,0.990000,0.989058,0.989529
5,0.018200,0.051873,0.988250,0.989519,0.988107,0.988812
6,0.012100,0.052559,0.989250,0.990939,0.988582,0.989759


✅ Training complete! Best binary model saved to /content/drive/MyDrive/experiments_binary


In [ ]:
# src/predict_binary.py
from transformers import logging
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer
from datasets import Dataset
import os

os.environ["WANDB_DISABLED"] = "true"
logging.set_verbosity_error()

def main():
    # --- Step 1: Paths ---
    model_dir = "/content/drive/MyDrive/experiments_binary/checkpoint-7500"
    test_path = "/content/drive/MyDrive/test.parquet"
    output_csv = "/content/drive/MyDrive/taska.csv"

    # --- Step 2: Load test data ---
    print(f"📂 Loading test data from {test_path} ...")
    test_df = pd.read_parquet(test_path)
    test_df = test_df.reset_index(drop=True)
    test_df["ID"] = test_df.index

    # --- Step 3: Load tokenizer and model ---
    print(f"🚀 Loading model and tokenizer from {model_dir} ...")
    tokenizer = AutoTokenizer.from_pretrained(model_dir)
    model = AutoModelForSequenceClassification.from_pretrained(model_dir)

    # --- Step 4: Tokenize test data ---
    print("🔡 Tokenizing test data ...")
    def preprocess(batch):
        return tokenizer(batch["code"], truncation=True, padding="max_length", max_length=256)

    test_ds = Dataset.from_pandas(test_df)
    test_tokenized = test_ds.map(preprocess, batched=True)

    # --- Step 5: Initialize Trainer for prediction ---
    trainer = Trainer(model=model)

    # --- Step 6: Predict ---
    print("🤖 Running predictions ...")
    preds = trainer.predict(test_tokenized)
    logits = preds.predictions
    y_pred = torch.argmax(torch.tensor(logits), dim=1).numpy()

    # --- Step 7: Save ONLY ID & predicted label ---
    submission = pd.DataFrame({
        "ID": test_df["ID"],
        "Label": y_pred
    })

    submission.to_csv(output_csv, index=False)
    print(f"✅ Predictions saved to {output_csv} (shape: {submission.shape})")

if __name__ == "__main__":
    main()

📂 Loading test data from /content/drive/MyDrive/test.parquet ...
🚀 Loading model and tokenizer from /content/drive/MyDrive/experiments_binary/checkpoint-7500 ...
🔡 Tokenizing test data ...


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

🤖 Running predictions ...
✅ Predictions saved to /content/drive/MyDrive/taska.csv (shape: (1000, 2))


In [ ]:
import pandas as pd

df = pd.read_parquet("/content/drive/MyDrive/test.parquet")
print(df.columns.tolist())


['code', 'generator', 'label', 'language']


In [ ]:
# src/predict_binary.py
from transformers import logging
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer
from datasets import Dataset
import os

os.environ["WANDB_DISABLED"] = "true"
logging.set_verbosity_error()

def main():
    # --- Step 1: Paths ---
    model_dir = "/content/drive/MyDrive/experiments_binary/checkpoint-7500"
    test_path = "/content/drive/MyDrive/test.parquet"
    output_csv = "/content/drive/MyDrive/taska.csv"

    # --- Step 2: Load test data ---
    print(f"📂 Loading test data from {test_path} ...")
    test_df = pd.read_parquet(test_path)
    test_df["ID"] = test_df.index

    # --- Step 3: Load tokenizer and model ---
    print(f"🚀 Loading model and tokenizer from {model_dir} ...")
    tokenizer = AutoTokenizer.from_pretrained(model_dir)
    model = AutoModelForSequenceClassification.from_pretrained(model_dir)

    # --- Step 4: Tokenize test data ---
    print("🔡 Tokenizing test data ...")
    def preprocess(batch):
        return tokenizer(batch["code"], truncation=True, padding="max_length", max_length=256)

    test_ds = Dataset.from_pandas(test_df)
    test_tokenized = test_ds.map(preprocess, batched=True)

    # --- Step 5: Initialize Trainer for prediction ---
    trainer = Trainer(model=model)

    # --- Step 6: Predict ---
    print("🤖 Running predictions ...")
    preds = trainer.predict(test_tokenized)
    logits = preds.predictions
    y_pred = torch.argmax(torch.tensor(logits), dim=1).numpy()

    # --- Step 7: Save ONLY ID & predicted label ---
    submission = pd.DataFrame({
        "ID": test_df["ID"],
        "Label": y_pred
    })

    submission.to_csv(output_csv, index=False)
    print(f"✅ Predictions saved to {output_csv} (shape: {submission.shape})")

if __name__ == "__main__":
    main()

📂 Loading test data from /content/drive/MyDrive/test.parquet ...
🚀 Loading model and tokenizer from /content/drive/MyDrive/experiments_binary/checkpoint-7500 ...
🔡 Tokenizing test data ...


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

🤖 Running predictions ...


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


✅ Predictions saved to /content/drive/MyDrive/taska.csv (shape: (1000, 2))
